# Sparse Zonal Harmonic Factorization

In [ ]:
import numpy as np
import sph_harm as sh
import sh_tools

lobe_dirs = np.array([
[0.0000, 0.0000],
[1.5708, 1.5708], [0.0000, 0.0000], [1.5708, 0.0000], 
[1.5708, 1.5708], [0.9553,-2.3562], [3.1416, 2.3562], [0.9553, 0.7854], [2.1863, 2.3562], 
[3.1416, 2.6180], [1.5708,-2.6180], [1.5708, 1.5708], [2.0344,-3.1416], [2.0344,-1.5708], [2.0344,-0.5236], [2.0344, 1.5708],
[1.5708, 0.7854], [1.1832, 0.0000], [1.5708,-3.1416], [1.1832, 0.7854], [3.1416, 0.0000], [1.5708, 1.5708], [1.5708, 0.3927], [2.2845,-1.5708], [0.8571,-3.1416],
[0.0000, 0.0000], [1.5708, 1.5708], [2.1863, 1.5708], [2.1863,-2.7489], [1.5708,-2.3562], [1.5708,-2.7489], [1.5708,-0.7854], [0.6997, 1.5708], [0.6997,-2.3562], [0.9553, 1.5708], [1.5708, 0.0000],
[1.5708, 0.7854], [1.0213,-2.6180], [2.1203,-1.5708], [1.5708,-1.5708], [3.1416, 1.5708], [1.5708, 0.5236], [2.1203, 1.5708], [1.8241, 1.5708], [0.5913,-0.3142], [1.8241,-1.5708], [2.1203,-3.1416], [1.5708, 0.3927], [2.3389,-1.5708],
[1.5708,-0.5236], [2.0719, 2.6180], [0.6928, 1.5708], [1.5708,-1.5708], [3.1416,-0.3927], [0.6928,-1.5708], [1.7989,-3.1416], [2.0053, 1.5708], [1.8518,-3.1416], [2.0053,-1.5708], [0.6928,-2.3562], [2.2040,-1.5708], [0.8755, 0.0000], [2.2040, 1.5708], [0.6928, 2.6180],
])


In [ ]:
def eq_D_l(l):
    return np.sqrt(4*np.pi/(2*l+1))


def eq_Y_l(l):
    matrix_size = 2*l+1
    dirs = lobe_dirs[(l)**2 : (l+1)**2]
    
    Y_l = np.zeros([matrix_size, matrix_size])
    for row in np.arange(0, matrix_size):
        w = dirs[row]
        for column in np.arange(0, matrix_size):
            m = column - l
            Y_l[row][column] = sh.sph_harm(m, l, w[0], w[1])
    return Y_l


def eq_Y(N):
    start_band = 0
    matrix_size = N**2 - start_band**2
    Y_l = np.zeros([matrix_size, matrix_size])

    # lobe sharing
    dirs = lobe_dirs[(N-1)**2 : (N)**2]

    for l in np.arange(start_band, N):
        diagonal_matrix_offset = l**2 - start_band**2
        diagonal_matrix_size = 2*l + 1
        for row in np.arange(0, diagonal_matrix_size):
            w = dirs[row]
            for column in np.arange(0, diagonal_matrix_size):
                m = column - l
                Y_l[row+diagonal_matrix_offset][column+diagonal_matrix_offset] = sh.sph_harm(m, l, w[0], w[1])
    return Y_l


def eq_Y_R(N, rot):
    start_band = 0
    matrix_size = N**2 - start_band**2
    Y_l = np.zeros([matrix_size, matrix_size])

    # lobe sharing
    dirs = lobe_dirs[(N-1)**2 : (N)**2]

    for l in np.arange(start_band, N):
        diagonal_matrix_offset = l**2 - start_band**2
        diagonal_matrix_size = 2*l + 1
        for row in np.arange(0, diagonal_matrix_size):
            w = dirs[row]
            theta, phi = w[0], w[1]
            x, y, z = spherical_dir(theta, phi)
            xyz = rot.apply([x, y, z])
            theta, phi = spherical_coord(xyz[0], xyz[1], xyz[2])
            
            for column in np.arange(0, diagonal_matrix_size):
                m = column - l
                Y_l[row+diagonal_matrix_offset][column+diagonal_matrix_offset] = sh.sph_harm(m, l, theta, phi)
    return Y_l


def eq_A_l_hat(l):
    A_hat = np.linalg.inv(eq_Y_l(l))
    return A_hat


def eq_A_hat(N):
    A_hat = np.linalg.inv(eq_Y(N))
    return A_hat


def print_matrix(m, N):
    for l in np.arange(N):
        offset = l**2
        for row in np.arange(2*l+1):
            linestr = ""
            for col in np.arange(2*l+1):
                linestr += "{:10.6f}".format(m[offset+row][offset+col]) + " "
            print(linestr)
        

In [ ]:
def spherical_dir(theta, phi):
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    return x, y, z

def spherical_coord(x, y, z):
    norm = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z/norm)
    phi = np.arctan2(y, x)
    return theta, phi

def meshgrid_spherical_coord(numsamples):
    theta = np.linspace(0, np.pi, num=numsamples)
    phi = np.linspace(0, 2*np.pi, num=numsamples*2)
    theta, phi = np.meshgrid(theta, phi)
    return theta, phi

# define a helper function that apply a rotation to a spherical function
def rotate_spherical_func(func, rotation):
    def func_r(theta, phi):
        x, y, z = spherical_dir(theta, phi)
        rot_matrix = rotation.as_dcm().transpose()
        x_p = rot_matrix[0][0]*x+rot_matrix[0][1]*y+rot_matrix[0][2]*z
        y_p = rot_matrix[1][0]*x+rot_matrix[1][1]*y+rot_matrix[1][2]*z
        z_p = rot_matrix[2][0]*x+rot_matrix[2][1]*y+rot_matrix[2][2]*z
        theta_p = np.arccos(z_p)
        phi_p = np.arctan2(y_p, x_p)
        return func(theta_p, phi_p)
    return func_r

In [ ]:
# define a function that we want to rotate
def func(theta, phi):
    return np.maximum(0, 5*np.cos(theta)-4) + np.maximum(0, -4*np.sin(theta-np.pi)*np.cos(phi-2.5+np.pi)-3)

In [ ]:
from scipy.spatial.transform import Rotation as R

# define a rotation that we want to apply to that function
rotation = R.from_euler('z', 44, degrees=True)

In [ ]:
# rotate the function and project into SH
func_rot = rotate_spherical_func(func, rotation)
sh_band = 3
sh_coeffs_rot = sh_tools.sh_projection(func_rot, sh_band, 256)
sh_tools.print_sh_coeffs(sh_band, sh_coeffs_rot)

In [ ]:
# project into Rotated Zonal Harmonic Basis
sh_coeffs = sh_tools.sh_projection(func, sh_band, 256)
A_hat = eq_A_hat(sh_band)
Z_hat = A_hat.transpose().dot(sh_coeffs)

# rotate in RZHB
Y_R = eq_Y_R(sh_band, rotation)
sh_coeffs_r = Y_R.transpose().dot(Z)
sh_tools.print_sh_coeffs(sh_band, sh_coeffs_r)

In [ ]:
# compare the results
sh_coeffs_r - sh_coeffs_rot

In [ ]:
# signal-tailored rotation

inv_rotation = rotation.inv()
Y_invR = eq_Y_R(sh_band, inv_rotation)
sh_coeffs_tilde = Y_invR.dot(sh_coeffs)
sh_coeffs_rot_str = A_hat.dot(sh_coeffs_tilde)
sh_tools.print_sh_coeffs(sh_band, sh_coeffs_rot_str)

In [ ]:
# compare the results
sh_coeffs_r - sh_coeffs_rot_str